In [220]:
# Libraries
import pandas as pd
import numpy as np
import ast

# Configure the notebook
pd.set_option('display.max_columns', None)

In [221]:
# Load data
df = pd.read_csv('../data/rent_Valencia.csv')
df.head()

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,country,latitude,longitude,showAddress,url,description,hasVideo,status,newDevelopment,hasLift,parkingSpace,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,district,neighborhood,labels,newDevelopmentFinished
0,97693036,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2203006-1,30,11,1500.0,penthouse,rent,80.0,True,3,2,Avenida de la Tarongina,València,Tavernes de la Valldigna,es,39.101077,-0.224147,False,https://www.idealista.com/inmueble/97693036/,Ático en alquiler en Tavernes de la Valldigna ...,True,good,False,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",19.0,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Tavernes de la Valldigna', 'titl...",False,False,False,False,False,NaN,NaN,NaN,NaN
1,98039610,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2206084,18,3,1600.0,flat,rent,193.0,True,4,1,Pinedo,València,Pinedo,es,39.420725,-0.342421,False,https://www.idealista.com/inmueble/98039610/,Piso en alquiler de 4 habitaciones en Pinedo A...,False,good,False,True,NaN,8.0,{'typology': 'flat'},"{'subtitle': 'Pinedo', 'title': 'Piso'}",False,False,False,False,False,NaN,NaN,NaN,NaN
2,98042660,https://img3.idealista.com/blur/WEB_LISTING/0/...,IM648,9,3,850.0,flat,rent,110.0,True,3,1,Calle Benasal,València,València,es,39.463590,-0.327468,False,https://www.idealista.com/inmueble/98042660/,"SOLO ALQUILERES, les ofrece este magnifico pis...",False,good,False,False,NaN,8.0,{'typology': 'flat'},"{'subtitle': 'El Cabanyal-El Canyamelar, Valèn...",False,False,False,False,False,Poblats Marítims,El Cabanyal-El Canyamelar,NaN,NaN
3,98042528,https://img3.idealista.com/blur/WEB_LISTING/0/...,V1134,29,3,950.0,flat,rent,101.0,True,3,2,barrio El Carme,València,València,es,39.479151,-0.380205,False,https://www.idealista.com/inmueble/98042528/,Referencia: V1134 Acogedora vivienda en una...,False,good,False,True,NaN,9.0,{'typology': 'flat'},"{'subtitle': 'El Carme, València', 'title': 'P...",False,False,True,False,False,Ciutat Vella,El Carme,NaN,NaN
4,98042519,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2206086,18,bj,900.0,flat,rent,45.0,True,1,1,Calle de Murillo,València,València,es,39.474174,-0.382997,False,https://www.idealista.com/inmueble/98042519/,Piso en alquiler de 1 habitación y terraza en ...,False,good,False,True,NaN,20.0,{'typology': 'flat'},"{'subtitle': 'El Pilar, València', 'title': 'P...",False,False,False,False,False,Ciutat Vella,El Pilar,NaN,NaN


**First**: Drop the evident unneccessary columns (like 'thumbnail' and 'description')

In [222]:
columns_to_drop = ['thumbnail', 'externalReference', 'operation', 'address', 
'country', 'url', 'description']
df.drop(columns_to_drop, axis=1, inplace=True)

**Second**: See and correct the poorly formatted data in the dataset.

In [223]:
# parkingSpace
df['parkingSpace'].value_counts()

{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}                                   701
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 100.0}       19
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 50.0}        16
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 150.0}       12
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 130.0}       10
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 80.0}         7
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 120.0}        5
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 90.0}         2
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 60.0}         1
{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkin

In [224]:
# replace all the ' to " in the parkingSpace column
df['parkingSpace'] = df['parkingSpace'].str.replace('\'', '"')

# convert the string to a dictionary
df['parkingSpace'] = df['parkingSpace'].apply(
    lambda x: ast.literal_eval(x) 
    if type(x) == str else x
    )

# get the hasParkingSpace of the dict of the parkingSpace column
df['hasParkingSpace'] = df['parkingSpace'].apply(
    lambda x: x['hasParkingSpace'] if type(x) == dict else False
    )

# get isParkingSpaceIncludedInPrice 
df['isParkingSpaceIncludedInPrice'] = df['parkingSpace'].apply(
    lambda x: x['isParkingSpaceIncludedInPrice'] if type(x) == dict else False
    )

# get the parkingSpacePrice 
df['parkingSpacePrice'] = df['parkingSpace'].apply(
    lambda x: x['parkingSpacePrice'] 
    if type(x) == dict and 'parkingSpacePrice' in x else 0
    )

# drop the parkingSpace column
df.drop(columns=['parkingSpace'], inplace=True)

In [225]:
# detailedType
df['detailedType'].value_counts()

{'typology': 'flat'}                                           2047
{'typology': 'chalet', 'subTypology': 'independantHouse'}       158
{'typology': 'flat', 'subTypology': 'penthouse'}                127
{'typology': 'flat', 'subTypology': 'duplex'}                    79
{'typology': 'flat', 'subTypology': 'studio'}                    64
{'typology': 'chalet', 'subTypology': 'terracedHouse'}           50
{'typology': 'chalet', 'subTypology': 'semidetachedHouse'}       36
{'typology': 'chalet'}                                           22
{'typology': 'countryHouse', 'subTypology': 'casaDePueblo'}      15
{'typology': 'countryHouse', 'subTypology': 'cortijo'}            1
{'typology': 'countryHouse', 'subTypology': 'countryHouse'}       1
Name: detailedType, dtype: int64

In [226]:
# suggestedTexts
df['suggestedTexts'].value_counts()

{'subtitle': 'El Cabanyal-El Canyamelar, València', 'title': 'Piso'}                32
{'subtitle': 'Sant Francesc, València', 'title': 'Piso'}                            30
{'subtitle': 'La Patacona, Alboraya', 'title': 'Piso en Avenida Mare Nostrum'}      25
{'subtitle': 'El Pla del Remei, València', 'title': 'Piso'}                         22
{'subtitle': 'La Roqueta, València', 'title': 'Piso'}                               18
                                                                                    ..
{'subtitle': 'El Pla del Remei, València', 'title': 'Piso en Cirilo Amoros, 37'}     1
{'subtitle': 'Aiora, València', 'title': 'Piso en Calle de Peris Brell'}             1
{'subtitle': 'Torrefiel, València', 'title': 'Piso en Calle Gayano Lluch'}           1
{'subtitle': 'Na Rovella, València', 'title': "Piso en Calle d'Aielo de Rugat"}      1
{'subtitle': 'Centro Ciudad, Paterna', 'title': 'Piso'}                              1
Name: suggestedTexts, Length: 1715, dtype: 

In [227]:
# labels
df['labels'].value_counts()

[{'name': 'luxuryType', 'text': 'Lujo'}]                                            88
[{'name': 'apartamentoType', 'text': 'Apartamento'}]                                84
[{'name': 'villaType', 'text': 'Villa'}, {'name': 'luxuryType', 'text': 'Lujo'}]    48
[{'name': 'villaType', 'text': 'Villa'}]                                            29
[{'name': 'loftType', 'text': 'Loft'}]                                               6
Name: labels, dtype: int64

In [228]:
# drop detailedType, suggestedTexts, labels columns
df.drop(columns=['detailedType', 'suggestedTexts', 'labels'], inplace=True) 

**Third**: Done with the obvious, now let's clean the data.

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   propertyCode                   2600 non-null   int64  
 1   numPhotos                      2600 non-null   int64  
 2   floor                          2254 non-null   object 
 3   price                          2600 non-null   float64
 4   propertyType                   2600 non-null   object 
 5   size                           2600 non-null   float64
 6   exterior                       2600 non-null   bool   
 7   rooms                          2600 non-null   int64  
 8   bathrooms                      2600 non-null   int64  
 9   province                       2600 non-null   object 
 10  municipality                   2600 non-null   object 
 11  latitude                       2600 non-null   float64
 12  longitude                      2600 non-null   f

In [230]:
# ¿There are duplicated rows?
df.duplicated().sum()

0

In [231]:
# ¿How percentage of the data is missing?
(df.isnull().sum() / df.shape[0] * 100).sort_values(ascending=False)

newDevelopmentFinished           99.807692
neighborhood                     35.846154
district                         15.346154
floor                            13.307692
hasLift                          11.846154
propertyCode                      0.000000
newDevelopment                    0.000000
isParkingSpaceIncludedInPrice     0.000000
hasParkingSpace                   0.000000
topNewDevelopment                 0.000000
hasStaging                        0.000000
has360                            0.000000
has3DTour                         0.000000
hasPlan                           0.000000
priceByArea                       0.000000
status                            0.000000
numPhotos                         0.000000
hasVideo                          0.000000
showAddress                       0.000000
longitude                         0.000000
latitude                          0.000000
municipality                      0.000000
province                          0.000000
bathrooms  

In [232]:
# newDevelopmentFinished
df[df['newDevelopmentFinished'].notnull()]

,propertyCode,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,province,municipality,latitude,longitude,showAddress,hasVideo,status,newDevelopment,hasLift,priceByArea,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,district,neighborhood,newDevelopmentFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice
1799,97836088,34,NaN,1220.0,flat,125.0,False,4,2,València,València,39.4408,-0.38141,False,False,newdevelopment,True,True,10.0,True,False,False,False,False,Quatre Carreres,Malilla,False,False,False,0.0
2438,97992884,33,NaN,1090.0,flat,109.0,False,3,2,València,València,39.4380,-0.38271,False,False,newdevelopment,True,True,10.0,False,False,False,False,False,Quatre Carreres,Malilla,False,False,False,0.0
2561,97836087,33,NaN,1110.0,flat,108.0,False,3,2,València,València,39.4404,-0.38381,False,False,newdevelopment,True,True,10.0,False,False,False,False,False,Quatre Carreres,Malilla,False,False,False,0.0
2582,97836055,33,NaN,880.0,flat,103.0,False,3,2,València,València,39.4389,-0.38221,False,False,newdevelopment,True,True,9.0,False,False,False,False,False,Quatre Carreres,Malilla,False,False,False,0.0
2583,97836077,33,NaN,850.0,flat,106.0,False,3,2,València,València,39.4411,-0.38111,False,False,newdevelopment,True,True,8.0,False,False,False,False,False,Quatre Carreres,Malilla,False,False,False,0.0


In [233]:
df['newDevelopmentFinished'].value_counts()

False    5
Name: newDevelopmentFinished, dtype: int64

So we can input the rest of newDevelopmentFinished as True, because we have another column that is newDevelopment and if newDevelopment is True and newDevelopmentFinished is True, then we can say that the project is finished and new.

In [234]:
index = df[df['newDevelopmentFinished'].isnull()].index
df.loc[index, 'newDevelopmentFinished'] = True

Because we have longitude and latitude, we can drop neighborhood without any problem, so let's continue with floor.

In [235]:
# floor
df[df['floor'].isnull()]

,propertyCode,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,province,municipality,latitude,longitude,showAddress,hasVideo,status,newDevelopment,hasLift,priceByArea,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,district,neighborhood,newDevelopmentFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice
29,97948119,60,NaN,4000.0,chalet,437.0,False,5,5,València,Rocafort,39.526525,-0.411505,False,False,good,False,NaN,9.0,False,False,False,False,False,NaN,NaN,True,True,True,0.0
30,97993156,42,NaN,1900.0,flat,191.0,True,4,2,València,València,39.472811,-0.381010,False,False,good,False,NaN,10.0,False,False,False,False,False,Ciutat Vella,Sant Francesc,True,False,False,0.0
31,98009651,53,NaN,3700.0,chalet,533.0,False,5,6,València,Puçol,39.630025,-0.335612,False,False,good,False,NaN,7.0,False,False,False,False,False,Los Monasterios-Alfinach,NaN,True,False,False,0.0
32,98009662,22,NaN,1700.0,flat,134.0,False,3,2,València,València,39.451680,-0.364370,False,False,renew,False,True,13.0,False,False,False,False,False,Quatre Carreres,Ciutat de les Arts i de les Ciencies,True,True,True,0.0
33,98025184,34,NaN,3700.0,penthouse,215.0,True,4,3,València,València,39.464390,-0.372898,False,False,good,False,True,17.0,False,False,False,False,False,L'Eixample,Gran Vía,True,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2577,92657917,14,NaN,500.0,chalet,221.0,False,6,2,València,Gátova,39.770668,-0.521205,False,False,good,False,NaN,2.0,False,False,False,False,False,NaN,NaN,True,False,False,0.0
2579,97905034,15,NaN,850.0,chalet,140.0,False,5,2,València,Riba-Roja de Túria,39.546271,-0.568119,False,False,good,False,NaN,6.0,False,False,False,False,False,NaN,NaN,True,False,False,0.0
2582,97836055,33,NaN,880.0,flat,103.0,False,3,2,València,València,39.438900,-0.382210,False,False,newdevelopment,True,True,9.0,False,False,False,False,False,Quatre Carreres,Malilla,False,False,False,0.0
2583,97836077,33,NaN,850.0,flat,106.0,False,3,2,València,València,39.441100,-0.381110,False,False,newdevelopment,True,True,8.0,False,False,False,False,False,Quatre Carreres,Malilla,False,False,False,0.0


To unknown's values of floor we will replace them with the mode of the column group by propertyType.

In [236]:
floor_nan = df[df['floor'].isnull()]
floor_nan['propertyType'].value_counts()

chalet          223
flat             99
countryHouse     14
penthouse         4
duplex            4
studio            2
Name: propertyType, dtype: int64

So we need to know the mode floor of the chalet, flat, country house, etc.

In [237]:
property_type = floor_nan['propertyType'].value_counts().index.to_list()

for i in property_type:
    # get the mode of the propertyType column
    mode = df['floor'][df['propertyType'] == i].mode()[0]
    # get the index of the rows with the propertyType i and the floor is null
    index = df[(df['propertyType'] == i) & (df['floor'].isnull())].index
    # replace the nan with the mode
    df.loc[index, 'floor'] = mode

In [238]:
# hasLift
df[df['hasLift'].isnull()]

,propertyCode,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,province,municipality,latitude,longitude,showAddress,hasVideo,status,newDevelopment,hasLift,priceByArea,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,district,neighborhood,newDevelopmentFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice
29,97948119,60,bj,4000.0,chalet,437.0,False,5,5,València,Rocafort,39.526525,-0.411505,False,False,good,False,NaN,9.0,False,False,False,False,False,NaN,NaN,True,True,True,0.0
30,97993156,42,1,1900.0,flat,191.0,True,4,2,València,València,39.472811,-0.381010,False,False,good,False,NaN,10.0,False,False,False,False,False,Ciutat Vella,Sant Francesc,True,False,False,0.0
31,98009651,53,bj,3700.0,chalet,533.0,False,5,6,València,Puçol,39.630025,-0.335612,False,False,good,False,NaN,7.0,False,False,False,False,False,Los Monasterios-Alfinach,NaN,True,False,False,0.0
39,98039625,28,bj,3600.0,chalet,280.0,False,3,4,València,Náquera,39.643460,-0.434498,False,False,good,False,NaN,13.0,False,False,False,False,False,Corral Nou - Montros - Estepar,NaN,True,True,True,0.0
45,97922847,20,bj,1600.0,chalet,200.0,False,5,3,València,Alboraya,39.511046,-0.321505,False,True,good,False,NaN,8.0,False,False,False,False,False,Port Saplaya,NaN,True,True,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,96044690,13,3,1500.0,flat,130.0,False,5,2,València,Puerto de Sagunto,39.668024,-0.232089,False,False,good,False,NaN,12.0,False,False,False,False,False,Nuevo Centro,NaN,True,False,False,0.0
2544,95588506,10,1,500.0,flat,115.0,False,3,2,València,Puerto de Sagunto,39.662068,-0.227018,False,False,good,False,NaN,4.0,False,False,False,False,False,Avda de Abril - 9 de Octubre,NaN,True,False,False,0.0
2565,96605084,7,bj,300.0,countryHouse,108.0,False,3,2,València,Anna,39.022648,-0.645708,False,False,good,False,NaN,3.0,False,False,False,False,False,NaN,NaN,True,False,False,0.0
2577,92657917,14,bj,500.0,chalet,221.0,False,6,2,València,Gátova,39.770668,-0.521205,False,False,good,False,NaN,2.0,False,False,False,False,False,NaN,NaN,True,False,False,0.0


In [239]:
df[df['hasLift'].isnull()]['floor'].value_counts()

bj    278
1      16
3       6
2       5
4       3
Name: floor, dtype: int64

Because the majority of the values of floor are bj or 1, we will assume that rest of values are False.

In [240]:
index = df[df['hasLift'].isnull()].index
df.loc[index, 'hasLift'] = False

So lets drop all the direction columns.

In [241]:
df.columns

Index(['propertyCode', 'numPhotos', 'floor', 'price', 'propertyType', 'size',
       'exterior', 'rooms', 'bathrooms', 'province', 'municipality',
       'latitude', 'longitude', 'showAddress', 'hasVideo', 'status',
       'newDevelopment', 'hasLift', 'priceByArea', 'hasPlan', 'has3DTour',
       'has360', 'hasStaging', 'topNewDevelopment', 'district', 'neighborhood',
       'newDevelopmentFinished', 'hasParkingSpace',
       'isParkingSpaceIncludedInPrice', 'parkingSpacePrice'],
      dtype='object')

In [242]:
columns_to_drop = ['province', 'municipality', 'showAddress', 'district', 'neighborhood']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [243]:
df.isnull().sum()

propertyCode                     0
numPhotos                        0
floor                            0
price                            0
propertyType                     0
size                             0
exterior                         0
rooms                            0
bathrooms                        0
latitude                         0
longitude                        0
hasVideo                         0
status                           0
newDevelopment                   0
hasLift                          0
priceByArea                      0
hasPlan                          0
has3DTour                        0
has360                           0
hasStaging                       0
topNewDevelopment                0
newDevelopmentFinished           0
hasParkingSpace                  0
isParkingSpaceIncludedInPrice    0
parkingSpacePrice                0
dtype: int64

Nice! one problem off.. So let's continue with the types of the columns.

In [244]:
# type of the columns
df.dtypes

propertyCode                       int64
numPhotos                          int64
floor                             object
price                            float64
propertyType                      object
size                             float64
exterior                            bool
rooms                              int64
bathrooms                          int64
latitude                         float64
longitude                        float64
hasVideo                            bool
status                            object
newDevelopment                      bool
hasLift                           object
priceByArea                      float64
hasPlan                             bool
has3DTour                           bool
has360                              bool
hasStaging                          bool
topNewDevelopment                   bool
newDevelopmentFinished            object
hasParkingSpace                     bool
isParkingSpaceIncludedInPrice       bool
parkingSpacePric

Suspicious columns:
- floor
- hasLift
- newDevelopmentFinished

In [245]:
df['floor'].value_counts()

1     542
bj    402
2     376
3     369
4     282
5     210
6     120
7     118
8      73
10     22
9      20
en     19
11     17
14      9
12      5
17      5
13      4
18      3
15      3
22      1
Name: floor, dtype: int64

We can input 0 into 'bj' and 'en' because this is the ground floor and mezzanine.

In [246]:
# where floor is 'bj' or 'en' put 0
df.loc[df['floor'] == 'bj', 'floor'] = 0
df.loc[df['floor'] == 'en', 'floor'] = 0

In [247]:
df['hasLift'].value_counts()

True     1860
False     740
Name: hasLift, dtype: int64

In [248]:
df['newDevelopmentFinished'].value_counts()

True     2595
False       5
Name: newDevelopmentFinished, dtype: int64

In [249]:
# set floor type as int
df['floor'] = df['floor'].astype(int)

# set hasLift type as bool
df['hasLift'] = df['hasLift'].astype(bool)

# set newDevelopmentFinished type as bool
df['newDevelopmentFinished'] = df['newDevelopmentFinished'].astype(bool)

### So our data is clean :-)

In [250]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   propertyCode                   2600 non-null   int64  
 1   numPhotos                      2600 non-null   int64  
 2   floor                          2600 non-null   int32  
 3   price                          2600 non-null   float64
 4   propertyType                   2600 non-null   object 
 5   size                           2600 non-null   float64
 6   exterior                       2600 non-null   bool   
 7   rooms                          2600 non-null   int64  
 8   bathrooms                      2600 non-null   int64  
 9   latitude                       2600 non-null   float64
 10  longitude                      2600 non-null   float64
 11  hasVideo                       2600 non-null   bool   
 12  status                         2600 non-null   o

In [251]:
df.sample(10)

,propertyCode,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,latitude,longitude,hasVideo,status,newDevelopment,hasLift,priceByArea,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,newDevelopmentFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice
1319,96777496,17,2,750.0,flat,88.0,True,2,1,38.993811,-0.161580,True,good,False,True,9.0,True,False,False,False,False,True,False,False,0.0
399,95908294,23,2,900.0,flat,87.0,True,1,1,39.484336,-0.373092,False,good,False,False,10.0,False,False,False,False,False,True,False,False,0.0
1354,95230553,16,8,1100.0,flat,120.0,True,3,2,39.453293,-0.337023,False,good,False,True,9.0,False,False,False,False,False,True,True,True,0.0
198,96068088,30,3,2500.0,flat,307.0,True,5,4,39.478638,-0.376520,False,good,False,True,8.0,False,False,False,False,False,True,False,False,0.0
2038,84721008,19,4,550.0,flat,57.0,True,3,1,39.466572,-0.401780,False,good,False,False,10.0,False,False,False,False,False,True,False,False,0.0
1591,96304588,18,1,1550.0,flat,78.0,True,2,2,39.475556,-0.378340,False,good,False,False,20.0,False,False,False,False,False,True,False,False,0.0
2153,97258420,65,1,980.0,flat,115.0,True,3,3,39.465857,-0.375798,False,good,False,True,9.0,False,False,False,False,False,True,False,False,0.0
1745,97735585,28,0,2000.0,chalet,108.0,False,4,2,39.290736,-0.285854,False,good,False,False,19.0,False,False,False,False,False,True,False,False,0.0
2344,97773820,18,5,900.0,flat,91.0,True,2,1,39.471943,-0.392714,False,good,False,True,10.0,False,False,False,False,False,True,False,False,0.0
1442,96823857,22,1,2100.0,flat,99.0,True,3,2,38.995660,-0.163050,True,good,False,False,21.0,False,False,False,False,False,True,False,False,0.0


In [252]:
df.to_csv('../data/rent_Valencia_cleaned.csv', index=False)